In [17]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession

In [18]:
spark = SparkSession.builder.appName("test").getOrCreate()
print(spark.version)

4.0.0


25/08/19 20:03:35 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [19]:
spark = (SparkSession.builder
         .appName("market-basket-demo")
         .master("local[*]")       # usa todos los cores locales
         .getOrCreate())

spark

25/08/19 20:03:35 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [20]:
spark = SparkSession.builder.appName("read-csv").getOrCreate()

# Leer CSV con encabezado e inferir tipos
df_csv = (spark.read
          .option("header", True)          # la primera fila son nombres de columnas
          .option("inferSchema", True)
          .option("delimiter", ";")     # intenta detectar el tipo (int, float, string…)
          .csv("/Users/hectorrr/proyectos/basket market/dataset/Assignment-1_Data.csv"))
df_csv = df_csv.dropna(subset=["BillNo", "Itemname"])
df_csv.show(5)
df_csv.printSchema()

25/08/19 20:03:35 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+------+--------------------+--------+----------------+-----+----------+--------------+
|BillNo|            Itemname|Quantity|            Date|Price|CustomerID|       Country|
+------+--------------------+--------+----------------+-----+----------+--------------+
|536365|WHITE HANGING HEA...|       6|01.12.2010 08:26| 2,55|     17850|United Kingdom|
|536365| WHITE METAL LANTERN|       6|01.12.2010 08:26| 3,39|     17850|United Kingdom|
|536365|CREAM CUPID HEART...|       8|01.12.2010 08:26| 2,75|     17850|United Kingdom|
|536365|KNITTED UNION FLA...|       6|01.12.2010 08:26| 3,39|     17850|United Kingdom|
|536365|RED WOOLLY HOTTIE...|       6|01.12.2010 08:26| 3,39|     17850|United Kingdom|
+------+--------------------+--------+----------------+-----+----------+--------------+
only showing top 5 rows
root
 |-- BillNo: string (nullable = true)
 |-- Itemname: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- Price: string (null

In [21]:
from pyspark.sql import functions as F

tx = (df_csv.groupBy("BillNo")
        .agg(F.collect_set("Itemname").alias("items")))

tx.show(truncate=False)

+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:
from pyspark.ml.fpm import FPGrowth

# Ajusta estos umbrales según tu volumen y sparsity reales
min_support = 0.02     # proporción mínima de transacciones (20%)
min_conf    = 0.02     # confianza mínima de las reglas (30%)

fp = FPGrowth(itemsCol="items", minSupport=min_support, minConfidence=min_conf)
model = fp.fit(tx)

frequent_itemsets = model.freqItemsets  # [items, freq]
association_rules = model.associationRules  # [antecedent, consequent, confidence, (lift en versiones recientes)]
frequent_itemsets.show(truncate=False)
association_rules.show(truncate=False)


+-------------------------------------------------------------------+----+
|items                                                              |freq|
+-------------------------------------------------------------------+----+
|[WHITE HANGING HEART T-LIGHT HOLDER]                               |2202|
|[LUNCH BAG  BLACK SKULL.]                                          |1260|
|[LUNCH BAG  BLACK SKULL., LUNCH BAG RED RETROSPOT]                 |632 |
|[LUNCH BAG CARS BLUE]                                              |1132|
|[LUNCH BAG CARS BLUE, LUNCH BAG RED RETROSPOT]                     |545 |
|[LUNCH BAG CARS BLUE, LUNCH BAG  BLACK SKULL.]                     |518 |
|[LUNCH BAG CARS BLUE, LUNCH BAG SPACEBOY DESIGN]                   |478 |
|[POSTAGE]                                                          |1070|
|[RABBIT NIGHT LIGHT]                                               |979 |
|[JUMBO  BAG BAROQUE BLACK WHITE]                                   |925 |
|[JUMBO  BAG BAROQUE BLAC

+------------------------------------+------------------------------------+-------------------+------------------+--------------------+
|antecedent                          |consequent                          |confidence         |lift              |support             |
+------------------------------------+------------------------------------+-------------------+------------------+--------------------+
|[SPOTTY BUNTING]                    |[PARTY BUNTING]                     |0.4245960502692998 |5.181302526474644 |0.02340657165479018 |
|[DOTCOM POSTAGE]                    |[JUMBO BAG RED RETROSPOT]           |0.6850282485875706 |6.706904480357378 |0.024000395882818687|
|[PARTY BUNTING]                     |[SPOTTY BUNTING]                    |0.2856280193236715 |5.181302526474644 |0.02340657165479018 |
|[PAPER CHAIN KIT 50'S CHRISTMAS]    |[PAPER CHAIN KIT VINTAGE CHRISTMAS] |0.4768153980752406 |11.880993297539412|0.026969517022961204|
|[LUNCH BAG CARS BLUE]               |[LUNCH BAG

In [23]:
# Número total de transacciones
N = tx.count()

# Soportes por itemset
supp_df = (frequent_itemsets
           .withColumn("itemset", F.col("items"))
           .withColumn("support", F.col("freq")/F.lit(N))
           .select("itemset","support","freq"))

# helper para buscar soporte de un itemset exacto (=conjunto)
def join_support(df_rules, col_items, alias_col):
    return (df_rules
            .join(supp_df.withColumnRenamed("itemset","k")
                         .withColumnRenamed("support", alias_col),
                  on=F.array_sort(F.col(col_items)) == F.array_sort(F.col("k")),
                  how="left")
            .drop("k"))

# Empezamos de las reglas básicas
rules = association_rules

# A: antecedent, B: consequent
rules = join_support(rules, "antecedent", "support_A")
rules = join_support(rules, "consequent", "support_B")

# Soporte de A∪B: podemos inferirlo también con conf = supp(A∪B)/supp(A)
rules = (rules
         .withColumn("support_AB", F.col("confidence") * F.col("support_A"))
         .withColumn("lift_calc", F.col("confidence")/F.col("support_B"))
         .withColumn("leverage", F.col("support_AB") - F.col("support_A")*F.col("support_B"))
         .withColumn("conviction",
                     F.when(F.col("confidence") < 0.999999,
                            (1 - F.col("support_B")) / (1 - F.col("confidence")))
                      .otherwise(F.lit(float("inf"))))
        )

# Ordenar por lift o leverage para inspección
rules_ordered = rules.orderBy(F.desc("lift_calc"), F.desc("confidence"))
rules_ordered.select(
    "antecedent","consequent","support_A","support_B","support_AB",
    "confidence","lift_calc","leverage","conviction"
).show(truncate=False)

+------------------------------------------------------------------+-----------------------------------+--------------------+--------------------+--------------------+-------------------+------------------+--------------------+------------------+
|antecedent                                                        |consequent                         |support_A           |support_B           |support_AB          |confidence         |lift_calc         |leverage            |conviction        |
+------------------------------------------------------------------+-----------------------------------+--------------------+--------------------+--------------------+-------------------+------------------+--------------------+------------------+
|[GREEN REGENCY TEACUP AND SAUCER, ROSES REGENCY TEACUP AND SAUCER]|[PINK REGENCY TEACUP AND SAUCER]   |0.03617379255740301 |0.03642121931908155 |0.025484956452889944|0.7045143638850889 |19.343513947540593|0.02416746282055381 |3.261000410545145 |
|[PINK REGEN

In [24]:
pred = model.transform(tx)  # agrega columna 'prediction' con sugerencias
pred.show(truncate=False)

+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [25]:
df_nuevo=pred.toPandas()

In [26]:
import pandas as pd

In [28]:
df_nuevo.head()

,BillNo,items,prediction
0,536366,"[HAND WARMER UNION JACK, HAND WARMER RED POLKA...",[]
1,536374,[VICTORIAN SEWING BOX LARGE],[]
2,536386,"[JUMBO BAG BAROQUE BLACK WHITE, WHITE WIRE EG...","[JUMBO BAG SCANDINAVIAN BLUE PAISLEY, WHITE HA..."
3,536398,"[HAND WARMER RED RETROSPOT, STRAWBERRY CERAMIC...",[]
4,536460,"[VICTORIAN SEWING BOX MEDIUM, CHICK GREY HOT W...",[PACK OF 72 RETROSPOT CAKE CASES]


In [30]:
df_nuevo.to_excel("./predicciones_market_basket.xlsx")